In [26]:
import pandas as pd
import numpy as np
import torch
import numpy as np
from sklearn.preprocessing import normalize
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModel
from pythainlp.tokenize import word_tokenize


pip install --quiet transformers
pip install torch
pip install pandas
pip install pythainlp

In [10]:
# โหลดไฟล์ CSV (กรณีชื่อคอลัมน์ 'label')
df = pd.read_csv('dataset_truefakenews.csv')

In [11]:
# กำหนด mapping dictionary
label_map = {
    'ข่าวจริง': 'true',
    'คลังความรู้': 'true',
    'ข่าวปลอม': 'fake',
    'อาชญากรออนไลน์': 'fake',
    'ข่าวบิดเบือน': 'fake',
    'ข่าวอื่นๆ': None  # หรือจะลบออกเลย
}

In [12]:
# สร้างคอลัมน์ใหม่ 'label_binary' เป็น binary label จาก mapping
#df['label_binary'] = df['label'].map(label_map)

# ลบแถวที่เป็น None (ข่าวอื่นๆ)
#df = df.dropna(subset=['label_binary'])

# ถ้าต้องการ save เป็น CSV ใหม่
#df.to_csv('news_dataset_truefakenews.csv', index=False)

# ดูผลลัพธ์
print(df.head())

                  date                                         link  \
0  23/07/2568 10:00:39  https://www.antifakenewscenter.com/?p=74610   
1  23/07/2568 09:00:20  https://www.antifakenewscenter.com/?p=74581   
2  23/07/2568 08:00:51  https://www.antifakenewscenter.com/?p=74577   
3  23/07/2568 07:00:51  https://www.antifakenewscenter.com/?p=74573   
4  23/07/2568 06:30:23  https://www.antifakenewscenter.com/?p=74569   

                                               title  \
0  กระทรวงกลาโหม จัดประชุม GBC เซ็นสัญญายอมให้ทหา...   
1  OR ชวนร่วมเป็นเจ้าของกิจการ ลงทุนหุ้น IPO เริ่...   
2  กระทรวงยุติธรรม เปิดเพจเฟซบุ๊กชื่อ Technology ...   
3  หุ้น OKJ เปิดเทรดวันแรก ราคาเปิดพอร์ตเริ่มต้น ...   
4  กรุงไทย เปิดให้จองสิทธิ์ยืมเงิน 100,000 บาท ลง...   

                                             content  \
0  ก่อนแชร์ต่อ อย่าลืมเช็ก ตามที่กองบัญชาการกองทั...   
1  จากที่มีการโฆษณาระบุเปิดให้ลงทุนหุ้น IPO ข้างต...   
2  ระวังเพจปลอมแอบอ้างหน่วยงาน เพจเฟซบุ๊กชื่อ Tec...   
3  เช็กข้อมู

In [26]:
# Load Thai BERT model and tokenizer from monsoon-nlp
model_name = "monsoon-nlp/bert-base-thai"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()

# Add device configuration and move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
model = model.to(device)

Some weights of the model checkpoint at monsoon-nlp/bert-base-thai were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cpu


In [13]:
def thai_token_list(text):
    if pd.isna(text): return []
    return word_tokenize(text, engine='newmm')

df['title_tokens'] = df['title'].apply(thai_token_list)
df['content_tokens'] = df['content'].apply(thai_token_list)


In [14]:
# Show sample of tokenized data
print("Sample of tokenized titles:")
print(df[['title', 'title_tokens']].head())
print("\nSample of tokenized content:")
print(df[['content', 'content_tokens']].head())

# Print token statistics
print("\nToken statistics:")
print(f"Average tokens in titles: {df['title_tokens'].str.len().mean():.2f}")
print(f"Average tokens in content: {df['content_tokens'].str.len().mean():.2f}")

Sample of tokenized titles:
                                               title  \
0  กระทรวงกลาโหม จัดประชุม GBC เซ็นสัญญายอมให้ทหา...   
1  OR ชวนร่วมเป็นเจ้าของกิจการ ลงทุนหุ้น IPO เริ่...   
2  กระทรวงยุติธรรม เปิดเพจเฟซบุ๊กชื่อ Technology ...   
3  หุ้น OKJ เปิดเทรดวันแรก ราคาเปิดพอร์ตเริ่มต้น ...   
4  กรุงไทย เปิดให้จองสิทธิ์ยืมเงิน 100,000 บาท ลง...   

                                        title_tokens  
0  [กระทรวงกลาโหม,  , จัด, ประชุม,  , GBC,  , เซ็...  
1  [OR,  , ชวน, ร่วม, เป็น, เจ้าของกิจการ,  , ลงท...  
2  [กระทรวงยุติธรรม,  , เปิด, เพจ, เฟซบุ๊ก, ชื่อ,...  
3  [หุ้น,  , OKJ,  , เปิด, เทรด, วัน, แรก,  , ราค...  
4  [กรุงไทย,  , เปิด, ให้, จอง, สิทธิ์, ยืม, เงิน...  

Sample of tokenized content:
                                             content  \
0  ก่อนแชร์ต่อ อย่าลืมเช็ก ตามที่กองบัญชาการกองทั...   
1  จากที่มีการโฆษณาระบุเปิดให้ลงทุนหุ้น IPO ข้างต...   
2  ระวังเพจปลอมแอบอ้างหน่วยงาน เพจเฟซบุ๊กชื่อ Tec...   
3  เช็กข้อมูลให้ชัวร์ก่อนติดตาม เพจ “OH KAD” เป็น..

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

In [19]:


# โหลด tokenizer และ model ก่อน
model_name = "bert-base-multilingual-cased"  # ถ้าเป็นภาษาไทยอาจใช้ "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_bert_embedding(word_list):
    # รวมคำเป็น string
    text = ' '.join(word_list)
    # encode เป็น tensor
    inputs = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )
    with torch.no_grad():
        outputs = model(**inputs)
        # ใช้ [CLS] token เป็น embedding
        cls_emb = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return cls_emb

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [20]:
df['title_emb'] = df['title_tokens'].apply(get_bert_embedding)
df['content_emb'] = df['content_tokens'].apply(get_bert_embedding)

In [21]:
print(df['title_emb'].iloc[0].shape)  # ควรได้ (768,)

(768,)


เสร็จ embedding

In [27]:
use_content = True
model.eval()

title_emb = np.vstack([get_bert_embedding(w) for w in df['title_tokens']])

if use_content: content_emb = np.vstack([get_bert_embedding(w) for w in df['content_tokens']]) 
x_np = np.concatenate([title_emb, content_emb], axis=1) # concat title+content else: x_np = title_emb
x_np = normalize(x_np, norm='l2', axis=1)
labels_str = df['label'].astype(str) 
classes = sorted(labels_str.unique()) 
label2id = {c: i for i, c in enumerate(classes)} 
y_np = labels_str.map(label2id).values

print("Classes:", label2id)

Classes: {'กิจกรรม': 0, 'ข่าวจริง': 1, 'ข่าวจริิง': 2, 'ข่าวบิดเบือน': 3, 'ข่าวปลอม': 4, 'ข่าวอื่นๆ': 5, 'ข่่าวจริง': 6, 'ข่่าวบิดเบือน': 7, 'คลังความรู้': 8, 'อาชญากรรมออนไลน์': 9}


สร้างกราฟ kNN + edge weight (cosine similarity)

In [ ]:
N = x_np.shape[0] 
k = min(8, max(3, int(np.round(np.log2(max(N, 2))))))
 # สูตรง่ายสำหรับ N < 1000 nbrs = NearestNeighbors(n_neighbors=k+1, metric='cosine').fit(x_np) dist, idx = nbrs.kneighbors(x_np)
src = np.repeat(np.arange(N), k) 
dst = idx[:, 1:].reshape(-1) w = (1.0 - dist[:, 1:].reshape(-1)) # cosine similarity ในช่วง [0,1]
edge_index_np = np.vstack([src, dst]) 
edge_weight_np = w
edge_index_np_rev = edge_index_np[::-1, :] 
edge_index_np = np.concatenate([edge_index_np, edge_index_np_rev], axis=1) 
edge_weight_np = np.concatenate([edge_weight_np, edge_weight_np], axis=0)
edge_index = torch.tensor(edge_index_np, dtype=torch.long) 
edge_weight = torch.tensor(edge_weight_np, dtype=torch.float) 
edge_index, edge_weight = coalesce(edge_index, edge_weight, m=N, n=N, reduce='mean')